# UCLQ Quantum Energy Hackathon - JASTR



1.   Install Qiskit



In [ ]:
!pip install qiskit
!pip install qiskit-optimization

     |████████████████████████████████| 154 kB 4.2 MB/s 
     |████████████████████████████████| 1.6 MB 50.6 MB/s 
     |████████████████████████████████| 634 kB 15.9 MB/s 
  Created wheel for docplex: filename=docplex-2.22.213-py3-none-any.whl size=696882 sha256=a29a3b7d5bddc6645f8d21454df46d2fe928cdbcf23271096e0d2ff73f7e0de2
  Stored in directory: /root/.cache/pip/wheels/90/69/6b/1375c68a5b7ff94c40263b151c86f58bd72200bf0c465b5ba3
Successfully built docplex
  Attempting uninstall: networkx
    Found existing installation: networkx 2.6.3
    Uninstalling networkx-2.6.3:
      Successfully uninstalled networkx-2.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
%matplotlib inline
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize


# New Section

In [ ]:
from qiskit import BasicAer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np

In [ ]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var("x")
qubo.binary_var("y")
qubo.binary_var("z")
qubo.minimize(linear=[1, -2, 3], quadratic={("x", "y"): 1, ("x", "z"): -1, ("y", "z"): 2})
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [ ]:
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: 1.5
operator:
-1.75 * ZII
+ 0.25 * IZI
+ 0.5 * ZZI
- 0.5 * IIZ
- 0.25 * ZIZ
+ 0.25 * IZZ


In [ ]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x0 - 2 x1 + 3 x2 + [ 2 x0*x1 - 2 x0*x2 + 4 x1*x2 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1

Binaries
 x0 x1 x2
End



In [ ]:
# Renewable sources: solar, on-shore and off-shore wind turbines
number_of_renewable_sources = 3
# Locations: England, Northern Ireland, Scotland, Wales
number_of_locations = 4

# Calculate the number of qubits needed, 
# NOTE: IBMQ systems accesible have 5 qubits
nqubits = number_of_locations * number_of_renewable_sources

G = nx.complete_graph(nqubits)

total_cost = C = 61510030000.0 # units: £

# Calculate revenue for each energy source in all locations 
# dict{location: [energy_source]}
location_revenue_per_unit = {
    0: [2783.55, 336113.74, 153105454.50],
    1: [2332.31, 434451.22, 0.0],
    2: [1088.38, 1020438.50, 85798571.43],
    3: [3573.59, 768855.93, 140790000],
}

# Calculate cost for each energy source in all locations
# dict{location: [energy_source]}
location_cost_per_unit = {
    0: [300.92, 52370.72, 22696783.84],
    1: [273.66, 70226.14, 0.0],
    2: [116.17, 161395.70, 11626961.90],
    3: [382.92, 122708.47, 21872844.44],
}

# variable and a rough estimate, see spreadsheet for current figures.
# dict{location: [energy_source]}
location_unit_capacity = {
    0: [800000, 5000, 40],
    1: [30000, 1400, 0],
    2: [70000, 4000, 10],
    3: [60000, 1000, 5],
}

# lambda and gamma, the initial parameter guess for the cost function
lam, gam = -10.0, 0.1

# Calculate the total costs and revenues 
a_revs, b_costs = {}, {}

for N in range(number_of_locations):
    for R, source in enumerate(["s", "n", "f"]):
        num_units = location_unit_capacity[N][R]
        a_revs[f"l{N}_{source}"]=num_units * location_revenue_per_unit[N][R]
        b_costs[f"l{N}_{source}"]=num_units * location_cost_per_unit[N][R]

In [ ]:
constant = lam * C**2
linear = []
quadratic = {}
lam=0.1
gam =0.1
# create a QUBO
qubo = QuadraticProgram()
for k, a in a_revs.items():
  qubo.binary_var(k)
  for ki, b in b_costs.items():
    # add linear terms
    if k == ki:
      linear.append(lam*a**2 + gam* b - 2*lam*C*a)
      continue
    if (ki, k) in quadratic.keys():
      continue
    if lam*a*b+0.0 == 0.0:
      continue
    # add quadratic terms
    quadratic[(k, ki)]=2*lam*a*b

qubo.minimize(linear=linear, quadratic=quadratic)
print(qubo.export_as_lp_string())


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 26898717402455924736 l0_s - 20391935115244048384 l0_n
      - 71589563963954044928 l0_f - 860273178120730112 l1_s
      - 7445475540134757376 l1_n - 936667570516830976 l2_s
      - 48547690626807840768 l2_n - 10481331456626970624 l2_f
      - 2633142140914386432 l3_s - 9399356319873693696 l3_n
      - 8610442563439063040 l3_f + [ 233242428249600000 l0_s*l0_n
      + 808673698020249600 l0_s*l0_f + 7312764412800001 l0_s*l1_s
      + 87574131454656000 l0_s*l1_n + 7243376078400000 l0_s*l2_s
      + 575043840940800000 l0_s*l2_n + 103565535349584000 l0_s*l2_f
      + 20464837747200000 l0_s*l3_s + 109300851733920000 l0_s*l3_n
      + 97414649825539200 l0_s*l3_f + 610296072194716928 l0_n*l0_f
      + 5518853165304001 l0_n*l1_s + 66091117571258080 l0_n*l1_n
      + 5466486644612000 l0_n*l2_s + 433978498775344000 l0_n*l2_n
      + 78159632980930112 l0_n*l2_f + 15444560798496000 l0_n*l3_s
      

In [ ]:
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: -1.0299081119352223e+20
operator:
4.2138528026665344e+18 * ZIIIIIIIIIII
+ 4.5976742022264934e+18 * IZIIIIIIIIII
+ 4204266538415382.5 * ZZIIIIIIIIII
+ 1.2960179551034017e+18 * IIZIIIIIIIII
+ 1172468672435094.0 * ZIZIIIIIIIII
+ 1315529283921900.0 * IZZIIIIIIIII
+ 5.143290867040992e+18 * IIIZIIIIIIII
+ 4691647015156546.0 * ZIIZIIIIIIII
+ 5264105714180506.0 * IZIZIIIIIIII
+ 985619669159268.1 * IIZZIIIIIIII
+ 2.3849900704873206e+19 * IIIIZIIIIIII
+ 2.231989257108694e+16 * ZIIIZIIIIIII
+ 2.5043289412819e+16 * IZIIZIIIIIII
+ 4688955725040000.0 * IIZIZIIIIIII
+ 2.37291991215863e+16 * IIIZZIIIIIII
+ 4.601055139258306e+17 * IIIIIZIIIIII
+ 416604412553126.1 * ZIIIIZIIIIII
+ 467437056025100.1 * IZIIIZIIIIII
+ 87520118616000.02 * IIZIIZIIIIII
+ 442909347745270.1 * IIIZIZIIIIII
+ 2459237927524000.5 * IIIIZZIIIIII
+ 3.641907187909509e+18 * IIIIIIIZIIII
+ 3325939383139876.0 * ZIIIIIIZIIII
+ 3731759114708338.5 * IZIIIIIZIIII
+ 698712256882080.0 * IIZIIIIZIIII
+ 3535943447643963.0 * IIIZIIIZIIII

In [ ]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 26898717402455924736 x0 - 20391935115244048384 x1
      - 71589563963954044928 x2 - 860273178120730112 x3 - 7445475540134757376 x4
      - 936667570516830976 x6 - 48547690626807840768 x7
      - 10481331456626970624 x8 - 2633142140914386432 x9
      - 9399356319873693696 x10 - 8610442563439063040 x11 + [
      233242428249600000 x0*x1 + 808673698020249600 x0*x2
      + 7312764412800001 x0*x3 + 87574131454656000 x0*x4
      + 7243376078400000 x0*x6 + 575043840940800000 x0*x7
      + 103565535349584000 x0*x8 + 20464837747200000 x0*x9
      + 109300851733920000 x0*x10 + 97414649825539200 x0*x11
      + 610296072194716928 x1*x2 + 5518853165304001 x1*x3
      + 66091117571258080 x1*x4 + 5466486644612000 x1*x6
      + 433978498775344000 x1*x7 + 78159632980930112 x1*x8
      + 15444560798496000 x1*x9 + 82488005562755600 x1*x10
      + 73517635491666064 x1*x11 + 20111442565665604 x2*x3
      

In [ ]:
algorithm_globals.random_seed = 10598
quantum_instance = QuantumInstance(
    BasicAer.get_backend("statevector_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

In [ ]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [ ]:
exact_result = exact.solve(qubo)
print(exact_result)

optimal function value: -2.0416864889600066e+20
optimal value: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
status: SUCCESS
